# Lab 3.3 - Azure AI Search



## Step 1: Upload data into Azure Blob Storage

1. If you don't have one, create a new Azure Storage account
2. Inside that account, create a new container called `healthplans`
3. Inside that account, create a new container called `images`
   
![Storage Account](../img/search02.png)

4. Upload the contents of the `data/health-plan` folder into the `healthplans` container
5. Upload the contents of the `data/images` folder into the `images` container (you'll use this later)

## Step 2: Create a new Azure Search service

1. In the Azure Portal, create a new Azure AI Search service.
2. Choose France Central (or the region you used before)
3. Choose Basic or higher tier
4. You can leave everything else as default
5. Click on Review + Create

![AI Search service](../img/search01.png)

## Step 3: Import health plan data into Azure AI Search

1. In Azure AI Search, start the "Import and vectorize data" wizard

![Import and Vectorize](../img/search03.png)

2. Select the Azure Blob Storage account and the hotels container
3. Check `Enable deletion tracking` and click on Next
4. Choose the Azure OpenAI Service created before
5. Choose an embedding model which will be used to vectorize the data. If you don't have one yet, just go with `text-embedding-ada-002`
6. Click Next
7. Leave `Vectorize images` and `Extract text from images` unchecked and click Next
8. Leave `Enable semantic ranker` checked and click Next
9. Give it a friendly name in `Objects name prefix` (eg: healthplans) and click Create

## Step 4: Experiment with queries

1. Make sure the indexing job is complete (it may take a few minutes).
2. Type `contoso` and see the results

![Index](../img/search04.png)


In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

# Load the environment variables with dotenv
from dotenv import load_dotenv
load_dotenv()
import os

service_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
key = os.getenv("AZURE_AI_SEARCH_API_KEY")
index_name = "healthplans" # replace this if you named your index differently

# Create a client
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

print('Using endpoint', service_endpoint)
print('Using index', index_name)

In [ ]:
# the word aerobics is NOT in the documents
query = "pilates"

# Let's start with a simple keyword search
results = search_client.search(
    search_text=query
    )

for r in results:
    print(r["chunk"])

# As you can see, this doesn't produce any results, as we just did a keyword search

## Let's step into vectors

In [ ]:
# The deployment name of the embedding model used in Azure AI Search
embedding_deployment_name = "ada-embedding" # Change this if you used another one

from openai import AzureOpenAI
import json 

# Endpoint and API key can be found in Azure AI Studio -> Project Settings -> Project Properties -> Get API endpoints
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
# The name of the deployment to tbe used. Found in Azure AI Studio -> Deployments
AZURE_OPENAI_DEPLOYMENT_ID = os.getenv("AZURE_OPENAI_DEPLOYMENT_ID")

# Print the endpoint to verify it was loaded correctly
print('If you see some text below, the endpoint was loaded successfully.')
print(AZURE_OPENAI_ENDPOINT)

client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-02-01",
    azure_deployment=embedding_deployment_name
)


## Vector size

Vector size is determined by the embedding model. Different models will have different vector sizes. The vector size is important because it determines the number of dimensions in which the data is represented.

In [ ]:
# Let's see vectors in action

response = client.embeddings.create(input=[query], model=embedding_deployment_name)
embeddings = response.data[0].embedding

print("Vector length is ", len(embeddings))
print("Vector is ", embeddings)

In [ ]:
# Now let's do an actual search using just vectors
vector_query = VectorizedQuery(vector=embeddings, k_nearest_neighbors=3, fields="text_vector")

results = search_client.search(
    vector_queries=[vector_query],
    top=5
    )

# this will contain the chunks of the text that were used to generate the vectors
chunks = []
# store chunks and view results
for result in results:
    chunks.append(result["chunk"])
    #print(json.dumps(result, indent=2))

# Check our chunk length. They should be very similar
print('Found chunks:', len(chunks))
for chunk in chunks:
    print(f'Chunk: {chunk}')


## Why do we get results with vectors?

Even if `aerobics` is not in the text, the vector representation of the word is close to the vector representation of `gym` and `fitness`, which are in the text.

In [ ]:
# Now let's do the same query but make it hybrid by passing both vectors and query text 
vector_query = VectorizedQuery(vector=embeddings, k_nearest_neighbors=3, fields="text_vector")

# To do a hybrid search, just add the search_text parameter alongside vector_queries
results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    top=5
    )

# this will contain the chunks of the text that were used to generate the vectors
chunks = []
# store chunks and view results
for result in results:
    chunks.append(result["chunk"])
    #print(json.dumps(result, indent=2))

# Check our chunk length. They should be very similar
print('Found chunks:', len(chunks))
for chunk in chunks:
    print(f'Chunk: {chunk}')

## Getting meaningful answers with the help of GPT

Notice that pilates is not defined in the document, but yoga and wellness is. 
Using vectors, were we able to get a meaningful answer for the query related to pilates?

In [ ]:
# Let's now use our GPT deployment to give us meaningful answers

# Create a new client for the GPT model
gpt_client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-02-01"
)

# Create a prompt passing the search results

completion = gpt_client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT_ID,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that helps users find information about health plans and other HR info. Return the minimum information necessary to answer the user's question.",
        },
        {
            "role": "user",
            "content": f"Am I covered for {query} classes in any of the plans? \n\n {chunks}",
        },
    ]
)

print(completion.choices[0].message.content)


## Lab results

This was a 10.000 ft view of Azure AI Search, which is a very powerful end-to-end tool for intelligent search. Our main goal was for you to understand the importance of vectors and embeddings in AI applications.

In a real world scenario, you could use things like filters to narrrow down the search results even more, among many other features.

In [ ]:
# Although not relevant for this exapmle, it's a good practice to use semantic reranker to improve the search results
# Let's do the same process but enabling semantic reranking

# For that, we'll add 4 new parameters to the call
results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    top=3,
    query_type="semantic",
    semantic_configuration_name="healthplans-semantic-configuration",
    query_caption="extractive"
    )

chunks = []
# store chunks and view results
for result in results:
    chunks.append(result["chunk"])

# Now let's call our LLM and see the output
completion = gpt_client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT_ID,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that helps users find information about health plans and other HR info. Return the minimum information necessary to answer the user's question.",
        },
        {
            "role": "user",
            "content": f"Am I covered for {query} classes? \n\n {chunks}",
        },
    ]
)

print(completion.choices[0].message.content)

## Optional next steps

There are many labs and end-to-end examples available in the Azure AI Search documentation and in GitHub. Here's some you could try:

* https://github.com/Azure/azure-search-vector-samples/tree/main
* https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/basic-vector-workflow/azure-search-vector-python-sample.ipynb